# 5.1: 虚时间演化基态算法

目标：时间演化块消减算法 Time-Evolving Block Decimation (TEBD)

基本思想：MPS+虚时间演化 -> 哈密顿量基态 ($\max_{|v|=1}(v^{\dagger}Mv)$)

所需基础知识：
- 本征值分解及其对应的优化问题 ([1.6](./1-6.ipynb) 节)
- 量子门的作用与TensorPureState类 ([2.5](./2-5.ipynb) 节)
- 哈密顿量基态问题及其计算 ([2.8](./2-8.ipynb) 节)
- 矩阵乘积态及其中心正交形式、最优虚拟维数裁剪 ([4.1](4-1.ipynb)、[4.2](4-2.ipynb)、[4.3](4-3.ipynb) 节)

## 回顾：矩阵最大本征态的幂级数算法

> 详细见 [1.6](./1-6.ipynb) 节

**投影法**，用于求解 Hermitian Matrix 某个特定的本征值和本征向量


* **定义投影矩阵**: $\tau$ 一般取为一个小量；投影矩阵有不同的定义方法，求的是不同的本征向量，具体看证明
    * $\rho = e^{-\tau H}$
    * $\rho = e^{\tau H}$
    * $\rho = e^{\tau H^2}$
    * $\rho = e^{-\tau H^2}$
* $\rho$ 和 $H$ 具有相同的本征向量，本征值间为指数关系$e^{-\tau\Gamma}$，证明：
  假设 $H$ is Hermitian，那么用泰勒展开得

  $\rho = \sum_{n=0}^{\infty} \frac{(-\tau)^n}{n!} H^n = \sum_{n=0}^{\infty} \frac{(-\tau)^n}{n!} (U\Gamma U^{\dagger})^n = U (\sum_{n=0}^{\infty} \frac{(-\tau)^n}{n!} \Gamma^n) U^{\dagger}$

  $\rho$ 的本征值为 $\sum_{n=0}^{\infty} \frac{(-\tau)^n}{n!} \Gamma^n = e^{-\tau \Gamma}$
* 注意：$\rho$ 里的每个元素不是从 $H$ 的元素的简单指数，看上面的泰勒展开就知道了
* 投影属性：$\lim_{m\to\infty}\rho^m v = C U_{:,0}$, $C$ 是一个常数

## 虚时间演化方法

对于哈密顿量 $\hat{H}$，虚时演化算符：$\hat{U}(\beta) = e^{-\beta\hat{H}}$，其中 $\beta = 1/T$ 为倒温度；

对于任意不与基态正交的量子态 $|\varphi\rangle$，满足

$$\lim_{\beta\to\infty} \hat{U}(\beta)|\varphi\rangle = |\varphi_0\rangle/Z$$

基态 $|\varphi_0\rangle$ 满足本征方程

$$\hat{H}|\varphi_0\rangle = E_0|\varphi_0\rangle$$

其中 $E_0$ 为基态能量，满足

$$E_0 = \min_{\langle\varphi\rangle} \langle\varphi|\hat{H}|\varphi\rangle$$

根据热力学平衡理论，量子态在温度 $T$ 出现的概率满足

$$P(|\varphi\rangle;T) = \frac{1}{Z}e^{-\beta\langle\varphi|\hat{H}|\varphi\rangle} = \frac{1}{Z}\langle\varphi|e^{-\beta\hat{H}}|\varphi\rangle = \frac{1}{Z}\langle\varphi|\hat{U}(\beta)|\varphi\rangle$$

> TODO: 证明 $e^{-\beta\langle\varphi|\hat{H}|\varphi\rangle} = \langle\varphi|e^{-\beta\hat{H}}|\varphi\rangle$

> $\hat{U}(\beta)$ 也叫有限温密度算符或者有限温密度矩阵

其中 $Z$ 为配分函数，满足

$$Z = \sum_{\langle\varphi\rangle} e^{-\beta\langle\varphi|\hat{H}|\varphi\rangle} = \text{Tr}(e^{-\beta\hat{H}}) = \sum_{n} e^{-\beta E_n}$$

> TODO: 证明 $\sum_{\langle\varphi\rangle} e^{-\beta\langle\varphi|\hat{H}|\varphi\rangle} = \text{Tr}(e^{-\beta\hat{H}}) = \sum_{n} e^{-\beta E_n}$

可见，当系统处于热平衡态时，最概然量子态为基态；在不考虑基态简并的情况下，温度越低，基态的概率越接近于1。

> 证明：
>
> $$
> \begin{align}
> \lim_{\beta \to \infty} P(|\varphi_0); \beta) &= \lim_{\beta \to \infty} \frac{e^{-\beta E_0}}{\sum_n e^{-\beta E_n}} \\
> &= \lim_{\beta \to \infty} \frac{1}{\sum_n \frac{e^{-\beta E_n}}{e^{-\beta E_0}}} \\
> &= \lim_{\beta \to \infty} \frac{1}{1 + e^{-\beta(E_1-E_0)} + e^{-\beta(E_2-E_0)} + ...} \\
> &= 1
> \end{align}
> $$
>

处于某激发态 $|\varphi_m\rangle$ ($m > 0$) 的概率在零温极限下满足：

$$
\begin{align}
\lim_{\beta\to\infty} P(|\varphi_m\rangle; \beta) &= \lim_{\beta\to\infty} \frac{e^{-\beta E_m}}{\sum_n e^{-\beta E_n}} \\
&= \lim_{\beta\to\infty} \frac{1}{e^{-\beta(E_0-E_m)} + e^{-\beta(E_1-E_m)} + \cdots} \\
&= 0
\end{align}
$$

### 回顾: Trotter-Suzuki (TS) 分解

> 在 [3.8](3-8.ipynb) 中也讲了 TS 分解

TS分解的目的：为了方便计算虚时演化$\hat{U}(\beta)|\varphi\rangle$，将虚时演化算符$\hat{U}(\beta)$分解成局域算符的连乘

$$\hat{U}(\beta) = [\hat{U}(\beta/K)]^K$$

考虑一维链，设 $\tau = \beta/K$（Trotter切片长度），哈密顿量 $\hat{H} = \sum_{l=0}^{L-2} \hat{H}^{(l,l+1)}$，有

$$\hat{U}(\tau) = e^{-\tau\hat{H}} = e^{-\tau\sum_{l=0}^{L-2} \hat{H}^{(l,l+1)}} = \prod_{l=0}^{L-2} e^{-\tau\hat{H}^{(l,l+1)}} + O(\tau^2)$$

> $O(\tau^2)$ 叫 Trotter 误差
>
> Trotter 误差来源于 $e^{-\tau (A+B)} \approx e^{-\tau A}e^{-\tau B}$

> Trotter 误差证明：
> 1. 对 $e^{\tau(\hat{A}+\hat{B})}$, $e^{\tau\hat{A}}$ 和 $e^{\tau\hat{B}}$ 都使用泰勒展开
> 2. 消去一次项
> 3. 整理二次项，注意 $AB \neq BA$
> 4. $[\hat{A}, \hat{B}]$ 一般认为是 $10^0$ 量级
>
> $$
> \begin{align}
> e^{\tau(\hat{A}+\hat{B})} - e^{\tau\hat{A}}e^{\tau\hat{B}} &= \tau^2\left(\frac{1}{2}\hat{A}\hat{B} + \frac{1}{2}\hat{B}\hat{A} - \hat{A}\hat{B}\right) + O(\tau^3) \\
> &= \frac{\tau^2}{2}(\hat{B}\hat{A} - \hat{A}\hat{B}) + O(\tau^3) \\
> &\approx -\frac{\tau^2}{2}[\hat{A}, \hat{B}] \\
> &= O(\tau^2)
> \end{align}
> $$




由 $\lim_{\beta\to\infty} \hat{U}(\beta)|\varphi\rangle = |\varphi_0\rangle$，有：

$$|\varphi_0\rangle \approx \lim_{K\to\infty} \left(\prod_{l=0}^{L-2} e^{-\tau\hat{H}^{(l,l+1)}}\right)^K |\varphi\rangle$$

### 伪代码

1. 随机初始化量子态(归一化向量)：$|\varphi\rangle$；
2. 计算虚时长度为$\tau$的局域虚时演化算符：$e^{-\tau\hat{H}^{(p)}}$；
3. 演化量子态并归一化：$e^{-\tau\hat{H}}|\varphi\rangle \rightarrow |\varphi\rangle/Z$（$Z = \sqrt{|\langle\varphi|\varphi\rangle|}$）；
4. 若 $|\varphi\rangle$ 收敛：
   1. 若 $\tau$ 足够小，则返回计算结果，计算结束。
   2. 否则，减小 $\tau$ 并回到步骤2。

### 例子

以一维量子自旋链的哈密顿量为例：

$$\hat{H} = \sum_{l=0}^{L-2} (J^x \hat{S}_l^x \hat{S}_{l+1}^x + J^y \hat{S}_l^y \hat{S}_{l+1}^y + J^z \hat{S}_l^z \hat{S}_{l+1}^z)$$

> $\hat{S}_l^z \hat{S}_{l+1}^z$ 这种都是两个算符的直积

其中$\hat{S}_l^{\alpha}$ ($\alpha = x, y, z$)为自旋算符，满足：

$$
\hat{S}_l^{x}=\frac{1}{2}\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix},\ 
\hat{S}_l^{y}=\frac{1}{2}\begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix},\ 
\hat{S}_l^{z}=\frac{1}{2}\begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
$$

> 可以参考 [2.2](2-2.ipynb) 和 [2.8](2-8.ipynb) 中的 `heisenberg` 函数

* 海森堡模型：$J^x = J^y = J^z$
* XY 模型：$J^x = J^y, J^z = 0$
* 外加均匀磁场：$\hat{H} \rightarrow \hat{H} + \sum_{l=0}^{L-1} (h^x \hat{S}_l^x + h^y \hat{S}_l^y + h^z \hat{S}_l^z)$
* 横场 Ising 模型：$J^x = J^y = 0, h^x \neq 0$
* 周期边界：$\hat{H} \rightarrow \hat{H} + J^x \hat{S}_0^x \hat{S}_{L-1}^x + J^y \hat{S}_0^y \hat{S}_{L-1}^y + J^z \hat{S}_0^z \hat{S}_{L-1}^z$

In [1]:
import torch
from tensor_network.algorithms.calc_ground_state_linear_operator import calc_ground_state
from tensor_network.quantum_state.functional import calc_observation
from tensor_network.tensor_gates.hamiltonians import heisenberg
from tensor_network.tensor_gates.functional import apply_gate

In [2]:
delta = 1.0  # 各向异性参数
num_qubits = 4  # 自旋个数
interact_positions = torch.tensor([[0, 1], [1, 2], [2, 3]])  # 相互作用位置

device = torch.device("cpu")
dtype = torch.float32

In [3]:
hamiltonian = heisenberg(
    jx=delta, jy=delta, jz=delta, double_precision=dtype in [torch.float64, torch.complex128]
)

ground_state, ground_energy_linear_operator = calc_ground_state(
    hamiltonian, interact_positions, num_qubits
)

In [4]:
tau = 0.2  # 初始Trotter切片长度
time_it = 1000  # 演化次数
time_ob = 20  # 观测量计算间隔
e0_converge_limit = 1e-3  # 基态能收敛判据
tau_min = 1e-5  # 终止循环tau判据

In [5]:
imag_time_evo_op = torch.matrix_exp(-tau * hamiltonian.reshape(4, 4)).reshape(2, 2, 2, 2)

In [6]:
quantum_state = torch.randn(*([2] * num_qubits), dtype=dtype, device=device)
quantum_state = quantum_state / quantum_state.norm()

In [7]:
e0 = 1.0
inversed_temperature = 0.0

for t in range(time_it):
    for positions in interact_positions:
        quantum_state = apply_gate(
            quantum_state=quantum_state, gate=imag_time_evo_op, target_qubit=positions.tolist()
        )

    quantum_state = quantum_state / quantum_state.norm()
    inversed_temperature += tau

    if t % time_ob == 0:
        ground_energy = 0.0
        for positions in interact_positions:
            ground_energy += calc_observation(
                quantum_state, hamiltonian, qubit_idx=positions.tolist()
            )

        print(f"\nAt iteration {t}")
        print(
            f"  Inversed Temperature = {inversed_temperature}, Ground Energy={ground_energy.item()}"
        )
        if abs(ground_energy - e0) < e0_converge_limit * tau:
            tau *= 0.5
            imag_time_evo_op = torch.matrix_exp(-tau * hamiltonian.reshape(4, 4)).reshape(
                2, 2, 2, 2
            )
            print(f"  Tau is reduced to {tau} since the ground energy is converged")
        if tau < tau_min:
            print(f"  Tau is less than {tau_min}, terminating the imaginary time evolution")
            break
        e0 = ground_energy


At iteration 0
  Inversed Temperature = 0.2, Ground Energy=0.1573605239391327

At iteration 20
  Inversed Temperature = 4.200000000000001, Ground Energy=-1.5888646841049194

At iteration 40
  Inversed Temperature = 8.200000000000003, Ground Energy=-1.6158905029296875

At iteration 60
  Inversed Temperature = 12.199999999999989, Ground Energy=-1.6160210371017456
  Tau is reduced to 0.1 since the ground energy is converged

At iteration 80
  Inversed Temperature = 14.199999999999982, Ground Energy=-1.6160250902175903
  Tau is reduced to 0.05 since the ground energy is converged

At iteration 100
  Inversed Temperature = 15.199999999999996, Ground Energy=-1.6160253286361694
  Tau is reduced to 0.025 since the ground energy is converged

At iteration 120
  Inversed Temperature = 15.700000000000003, Ground Energy=-1.6160252094268799
  Tau is reduced to 0.0125 since the ground energy is converged

At iteration 140
  Inversed Temperature = 15.949999999999989, Ground Energy=-1.616025447845459

In [8]:
print(f"基态能量（线性算子法）= {ground_energy_linear_operator.item()}")
print(f"基态能量（虚时间演化）= {e0.item()}")

基态能量（线性算子法）= -1.6160253286361694
基态能量（虚时间演化）= -1.616025686264038
